# 3팀 project2 코드입니다.

In [106]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm

In [107]:
import pickle

def open_pickle(path: str):
    with open(path, 'rb') as f:
        target = pickle.load(f)
    return target

## 1. train, valid 데이터 셋 불러오기

In [108]:
train_path = r"./train.csv"
valid_path = r"./valid.csv"

train_df = pd.read_csv(train_path)
valid_df = pd.read_csv(valid_path)

In [109]:
# train_df.head() only Label == "Attack IP"
train_df_check = train_df[train_df["Label"] == "Attack_IP"]
train_df_check.head()


,Src IP,Src Port,Dst IP,Dst Port,Protocol,Flags,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd Header Length,Bwd Header Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
151,184.168.154.121,0,30.225.198.198,0,0,........,04:19:27,91806604,9,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.147583e+07,1.994660e+07,60095050.0,1279541.0,91806604.0,1.147583e+07,1.994660e+07,60095050.0,1279541.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,5.981982e+06,1.043562e+06,6719891.0,5244072.0,2.661421e+07,2.900580e+07,60095050.0,9091776.0,Attack_IP
152,184.168.161.142,0,30.225.198.198,0,0,........,04:19:26,78142570,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.112767e+06,6.729582e+06,30655548.0,159998.0,78142570.0,4.112767e+06,6.729582e+06,30655548.0,159998.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,8.266500e+06,4.895808e+06,13919693.0,5439843.0,1.066861e+07,1.117533e+07,30655548.0,5379867.0,Attack_IP
153,30.225.198.198,3004,112.21.63.135,6379,TCP,......S.,01:04:37,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29200,0,0,32,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,Attack_IP
154,184.168.153.125,0,30.225.198.198,0,0,........,04:20:41,30431417,17,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.901964e+06,2.442203e+06,7231862.0,60015.0,30431417.0,1.901964e+06,2.442203e+06,7231862.0,60015.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1.823946e+06,1.176619e+06,2655941.0,991950.0,7.039915e+06,2.714541e+05,7231862.0,6847968.0,Attack_IP
155,184.168.155.117,0,30.225.198.198,0,0,........,04:19:17,108830191,41,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.720755e+06,3.035440e+06,12159724.0,92010.0,108830191.0,2.720755e+06,3.035440e+06,12159724.0,92010.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,7.922295e+06,6.914584e+06,14783706.0,92010.0,8.044352e+06,2.165169e+06,12159724.0,5567966.0,Attack_IP


In [110]:
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [111]:
valid_df.head()

,Src IP,Src Port,Dst IP,Dst Port,Protocol,Flags,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd Header Length,Bwd Header Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,173.123.45.54,61650,30.239.23.122,53,TCP,...A...F,00:47:49,4454322,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2227161.0,3.146219e+06,4451874.0,2448.0,4451874.0,4451874.0,0.000000,4451874.0,4451874.0,0.0,0.0,0.0,0.0,0.0,40,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1024,4,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
1,21.122.91.165,35929,30.239.23.122,53,TCP,......S.,04:05:32,2988477,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1494238.5,7.149578e+05,1999790.0,988687.0,2988477.0,1494238.5,714957.787778,1999790.0,988687.0,0.0,0.0,0.0,0.0,0.0,120,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29200,0,0,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
2,173.123.45.54,64652,30.239.23.122,53,TCP,...A...F,02:07:05,5092931,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2546465.5,3.597473e+06,5090263.0,2668.0,5090263.0,5090263.0,0.000000,5090263.0,5090263.0,0.0,0.0,0.0,0.0,0.0,40,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1024,4,0,20,0.0,0.0,0.0,0.0,5090263.0,0.0,5090263.0,5090263.0,Benign_IP
3,21.122.91.166,42876,30.239.23.122,53,TCP,......S.,03:19:22,2993254,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1496627.0,7.145413e+05,2001884.0,991370.0,2993254.0,1496627.0,714541.301884,2001884.0,991370.0,0.0,0.0,0.0,0.0,0.0,120,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29200,0,0,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
4,173.123.45.54,62593,30.239.23.122,53,TCP,...A...F,00:53:13,4534770,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2267385.0,3.203341e+06,4532489.0,2281.0,4532489.0,4532489.0,0.000000,4532489.0,4532489.0,0.0,0.0,0.0,0.0,0.0,40,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1024,4,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP


## 2. feature 추가하기

도메인 지식을 활용하여 feature를 자유롭게 추가하시면 됩니다.

In [112]:
import ipaddress

train_flags =train_df[['Flags']]
valid_flags =valid_df[['Flags']]

train_src_ip = train_df[['Src IP']]
train_dst_ip = train_df[['Dst IP']]

valid_src_ip = valid_df[['Src IP']]
valid_dst_ip = valid_df[['Dst IP']]


def map_ip_to_class(ip):
    first_octet = int(ip.split('.')[0])
    if 1<= first_octet <= 126:
        return 1
    elif 128 <= first_octet <= 191:
        return 2
    elif 192 <= first_octet <= 223:
        return 3
    elif 224 <= first_octet <= 239:
        return 4
    else:
        return 5
    
train_src_ip['Class'] = train_src_ip['Src IP'].apply(map_ip_to_class)
train_dst_ip['Class'] = train_dst_ip['Dst IP'].apply(map_ip_to_class)

valid_src_ip['Class'] = valid_src_ip['Src IP'].apply(map_ip_to_class)
valid_dst_ip['Class'] = valid_dst_ip['Dst IP'].apply(map_ip_to_class)


def convert_to_int(ip):
	return int(ipaddress.IPv4Address(ip))

train_src_ip['Int'] = train_src_ip['Src IP'].apply(convert_to_int)
train_dst_ip['Int'] = train_dst_ip['Dst IP'].apply(convert_to_int)

valid_src_ip['Int'] = valid_src_ip['Src IP'].apply(convert_to_int)
valid_dst_ip['Int'] = valid_dst_ip['Dst IP'].apply(convert_to_int)

flag_mapping = {
    '.' : 0,  # 예시에 따라 .은 0에 매핑
    'A' : 1,  # 예시에 따라 A는 1에 매핑
    'F' : 2,
    'S' : 3,
    'R' : 4,
    'P' : 5,
    'C' : 6,
    'E' : 7,
    'U' : 8,
    # 다른 플래그에 대한 매핑도 필요한 경우 추가 가능
}

protocol_mapping = {
    '0' : 0,
    'TCP' : 1,
    'UDP' : -1
}

def encode_flags(flags_str):
	encoded = [flag_mapping[char] for char in flags_str if char in flag_mapping]
	res = ''
	for i in encoded:
		res += str(i)
	return res


	
train_flags['Encoded_Flags'] = train_flags['Flags'].apply(encode_flags)
train_protocol = train_df[['Protocol']].apply(lambda x: x.map(protocol_mapping))


valid_flags['Encoded_flags'] =valid_df['Flags'].apply(encode_flags)
valid_protocol = valid_df[['Protocol']].apply(lambda x: x.map(protocol_mapping))

#train_protocol.head()

train_src_ip.rename(columns={'Int':'CIDR'}, inplace=True)
train_dst_ip.rename(columns={'Int':'CIDR'}, inplace=True)
valid_src_ip.rename(columns={'Int':'CIDR'}, inplace=True)
valid_dst_ip.rename(columns={'Int':'CIDR'}, inplace=True)
train_src_ip.head()

/var/folders/7v/jspts5lx38l1dn20zwm8kz3m0000gn/T/ipykernel_47173/3125037583.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_src_ip['Class'] = train_src_ip['Src IP'].apply(map_ip_to_class)
/var/folders/7v/jspts5lx38l1dn20zwm8kz3m0000gn/T/ipykernel_47173/3125037583.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dst_ip['Class'] = train_dst_ip['Dst IP'].apply(map_ip_to_class)
/var/folders/7v/jspts5lx38l1dn20zwm8kz3m0000gn/T/ipykernel_47173/3125037583.py:29: SettingWithCopyWarning: 
A valu

,Src IP,Class,CIDR
0,21.122.91.166,1,360340390
1,21.122.91.166,1,360340390
2,21.122.91.165,1,360340389
3,173.123.45.117,2,2910530933
4,21.122.91.165,1,360340389


In [113]:
#train_expt = train_df[['Idle Mean','Flow IAT Mean','Src Port','Dst Port','Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
#			'Total Length of Fwd Packet', 'Total Length of Bwd Packet','Fwd Header Length', 'Bwd Header Length']]
# 'Idle Max', 'Flow IAT Max', 'Fwd IAT Max', 'Idle Mean', 'Flow Duration','Fwd IAT Total'
# 'Active Std','Fwd Packet Length Std','Bwd IAT Std','Bwd Packet Length Std'

train_expt = train_df[['Src Port','Flow IAT Mean','Flow IAT Max', 'Fwd Header Length', 'Dst Port','FWD Init Win Bytes','Fwd IAT Min','Flow IAT Min','Fwd Seg Size Min','Flow IAT Std','Flow Duration','Fwd IAT Mean', 'Total Fwd Packet', 'Total Bwd packets', 'Fwd IAT Max', 'Fwd IAT Std', 'Fwd IAT Total', 'Total Length of Bwd Packet','Bwd Header Length','Bwd Init Win Bytes','Fwd Act Data Pkts','Fwd Packet Length Max','Total Length of Fwd Packet','Packet Length Variance','Fwd Segment Size Avg','Bwd IAT Total','Packet Length Min','Bwd Segment Size Avg','Fwd Packet Length Min','Average Packet Size','Packet Length Max','Bwd Packet Length Max','Packet Length Std','Idle Std','Bwd IAT Mean','Bwd Packet Length Min','Bwd IAT Min','Bwd IAT Max']]
train_expt = pd.concat([train_expt, train_flags['Encoded_Flags'],train_protocol,train_src_ip[['Class','CIDR']],train_dst_ip[['Class','CIDR']]], axis=1)

train_list = train_expt.values.tolist()

#valid_expt = valid_df[['Idle Mean','Flow IAT Mean','Src Port','Dst Port','Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
			#'Total Length of Fwd Packet', 'Total Length of Bwd Packet','Fwd Header Length', 'Bwd Header Length']]
valid_expt = valid_df[['Src Port','Flow IAT Mean','Flow IAT Max', 'Fwd Header Length', 'Dst Port','FWD Init Win Bytes','Fwd IAT Min','Flow IAT Min','Fwd Seg Size Min','Flow IAT Std','Flow Duration','Fwd IAT Mean', 'Total Fwd Packet', 'Total Bwd packets', 'Fwd IAT Max', 'Fwd IAT Std', 'Fwd IAT Total', 'Total Length of Bwd Packet','Bwd Header Length','Bwd Init Win Bytes','Fwd Act Data Pkts','Fwd Packet Length Max','Total Length of Fwd Packet','Packet Length Variance','Fwd Segment Size Avg','Bwd IAT Total','Packet Length Min','Bwd Segment Size Avg','Fwd Packet Length Min','Average Packet Size','Packet Length Max','Bwd Packet Length Max','Packet Length Std','Idle Std','Bwd IAT Mean','Bwd Packet Length Min','Bwd IAT Min','Bwd IAT Max']]
valid_expt = pd.concat([valid_expt, valid_flags['Encoded_flags'],valid_protocol,valid_src_ip[['Class','CIDR']],valid_dst_ip[['Class','CIDR']]], axis=1)
valid_list = valid_expt.values.tolist()

#train_expt.head(2000)

In [114]:
print(len(valid_expt.columns))

44


In [115]:
train_y = [0 if i == "Benign_IP" else 1 for i in train_df['Label']]
valid_y = [0 if i == "Benign_IP" else 1 for i in valid_df['Label']]

### 스케일러 적용

In [116]:
# standardScaler 
from sklearn.preprocessing import StandardScaler

normalizer = StandardScaler()
train_X = []
valid_X = []

normalizer.fit(train_list)

train_X = normalizer.transform(train_list)
valid_X = normalizer.transform(valid_list)

In [117]:
#학습을 위한 데이터 타입 변환

train_X = train_X.astype(np.float32)
valid_X = valid_X.astype(np.float32)
train_y = np.array(train_y, dtype=np.float32)
valid_y = np.array(valid_y, dtype=np.float32)

# 3. 모델 학습 

In [126]:
# from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from sklearn.metrics import classification_report
# rfc = RandomForestClassifier(random_state = 108) # randomstate는 108로 고정 
# rfc.fit(train_X, train_y)
# y_pred = rfc.predict(valid_X)
from lightgbm import LGBMClassifier


rus = RandomUnderSampler(random_state=42)
train_X_resampled, train_y_resampled = rus.fit_resample(train_X, train_y)
# 1 try : 585
model = LGBMClassifier(random_state=108, n_estimators=1000, learning_rate=0.08903001761524029, max_depth=6, num_leaves=223, objective='binary', metric='binary_logloss')
model.fit(train_X_resampled, train_y_resampled)
# model.fit(train_X, train_y)
y_pred = model.predict(valid_X)


# lgbmc = LGBMClassifier(random_state=108, n_estimators = 1000, learning_rate = 0.08903001761524029, max_depth = 6, num_leaves = 223, objective='binary', metric='binary_logloss')
# lgbmc.fit(train_X, train_y)
# y_pred = lgbmc.predict(valid_X)


[LightGBM] [Info] Number of positive: 406136, number of negative: 406136
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.072384 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9295
[LightGBM] [Info] Number of data points in the train set: 812272, number of used features: 44
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

In [127]:
print(y_pred.shape)

(325560,)


플로우 기반 이상탐지 성능평가

In [128]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# 성능 평가를 위해 예측 값을 반올림
round_valid_y = valid_y.astype("int")
round_y_pred = np.round(y_pred)
print("confusion_matrix\n", confusion_matrix(round_valid_y, round_y_pred))
print(f"accuracy\t: {accuracy_score(round_valid_y, round_y_pred): .5f}")
print(f"precision\t: {precision_score(round_valid_y, round_y_pred): .5f}")
print(f"recall\t\t: {recall_score(round_valid_y, round_y_pred): .5f}")
print(f"F-1\t\t: {f1_score(round_valid_y, round_y_pred): .5f}")

confusion_matrix
 [[190156    117]
 [    87 135200]]
accuracy	:  0.99937
precision	:  0.99914
recall		:  0.99936
F-1		:  0.99925


# 4. IP voting

실제 성능평가는 IP기준으로 해야하기 때문에 flow 기반 예측결과를 이용하여 IP votiong을 진행합니다.

In [129]:
# 정상 외부 IP set (정답)
outer_benignIP_set = open_pickle("./outer_benign_IP_train.pkl")
# 악성 외부 IP set (정답)
outer_malIP_set = open_pickle("./outer_mal_IP_train.pkl")

In [130]:
#외부 IP의 악성, 정상 개수 확인
len(outer_benignIP_set), len(outer_malIP_set)

(590, 457)

In [131]:
# IP 탐지를 위해 dictionary형태로 변경
labelIP_dict = dict()

for ip in outer_benignIP_set:
    labelIP_dict[ip] = 0
for ip in outer_malIP_set:
    labelIP_dict[ip] = 1

In [132]:
# predictIP_dict = {}

# for idx, data in enumerate(valid_df.values):
#     if data[0] in labelIP_dict:
#         if data[0] not in predictIP_dict:
#             predictIP_dict[data[0]] = 0

#         if y_pred[idx] == 1:
#             predictIP_dict[data[0]] = 1

#     else:
#         if data[2] not in predictIP_dict:
#             predictIP_dict[data[2]] = 0

#         if y_pred[idx] == 1:
#             predictIP_dict[data[2]] = 1
from tqdm import tqdm

predictIP_dict = dict()
external_IP =list(labelIP_dict.keys())
external_IP_group = dict()

train_df.rename(columns={'Src IP':'src_ip','Dst IP':'dst_ip'}, inplace=True)
valid_df.rename(columns={'Src IP':'src_ip','Dst IP':'dst_ip'}, inplace=True)

# grouping
for row in tqdm(valid_df.itertuples()):
  src_ip = row.src_ip
  dst_ip = row.dst_ip

  if src_ip in external_IP:
    if src_ip in external_IP_group:
      external_IP_group[src_ip].append(row)
    else:
      external_IP_group[src_ip] = [row]
  else:
    if dst_ip in external_IP_group:
      external_IP_group[dst_ip].append(row)
    else:
      external_IP_group[dst_ip] = [row]

# if proportion 0.7
proportion = 0.55
for key, values in tqdm(external_IP_group.items()):
  predictIP_dict[key] = 0
  cnt = 0
  for value in values:
    if y_pred[value.Index] == 1:
      cnt += 1
      if cnt/float(len(values)) >= proportion:
          predictIP_dict[key] = 1
          break

325560it [00:04, 71179.80it/s] 
100%|██████████| 1047/1047 [00:00<00:00, 2601.45it/s]


In [133]:
pred_IP = []
label_IP = []

for k in predictIP_dict.keys():
    pred_IP.append(1 if predictIP_dict[k] == 1 else 0)
    label_IP.append(labelIP_dict[k])

print("confusion_matrix\n", confusion_matrix(label_IP, pred_IP))
print(f"accuracy\t: {accuracy_score(label_IP, pred_IP): .5f}")
print(f"precision\t: {precision_score(label_IP, pred_IP): .5f}")
print(f"recall\t\t: {recall_score(label_IP, pred_IP): .5f}")
print(f"F-1\t\t: {f1_score(label_IP, pred_IP): .5f}")

confusion_matrix
 [[589   1]
 [  0 457]]
accuracy	:  0.99904
precision	:  0.99782
recall		:  1.00000
F-1		:  0.99891


# 5. 제출 csv 생성 

In [134]:
# test.csv가 제공되면 실행하면 됩니다!
test_df = pd.read_csv("test.csv")

In [135]:
print(len(test_df))

325963


In [136]:
test_flags =test_df[['Flags']]
test_src_ip = test_df[['Src IP']]
test_dst_ip = test_df[['Dst IP']]
test_src_ip['Class'] = test_src_ip['Src IP'].apply(map_ip_to_class)
test_dst_ip['Class'] = test_dst_ip['Dst IP'].apply(map_ip_to_class)
test_src_ip['Int'] = test_src_ip['Src IP'].apply(convert_to_int)
test_dst_ip['Int'] = test_dst_ip['Dst IP'].apply(convert_to_int)
test_flags['Encoded_flags'] =test_df['Flags'].apply(encode_flags)
test_protocol = test_df[['Protocol']].apply(lambda x: x.map(protocol_mapping))
test_src_ip.rename(columns={'Int':'CIDR'}, inplace=True)
test_dst_ip.rename(columns={'Int':'CIDR'}, inplace=True)

/var/folders/7v/jspts5lx38l1dn20zwm8kz3m0000gn/T/ipykernel_47173/1511148250.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_src_ip['Class'] = test_src_ip['Src IP'].apply(map_ip_to_class)
/var/folders/7v/jspts5lx38l1dn20zwm8kz3m0000gn/T/ipykernel_47173/1511148250.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dst_ip['Class'] = test_dst_ip['Dst IP'].apply(map_ip_to_class)
/var/folders/7v/jspts5lx38l1dn20zwm8kz3m0000gn/T/ipykernel_47173/1511148250.py:6: SettingWithCopyWarning: 
A value is tr

In [137]:
test_expt = test_df[['Src Port','Flow IAT Mean','Flow IAT Max', 'Fwd Header Length', 'Dst Port','FWD Init Win Bytes','Fwd IAT Min','Flow IAT Min','Fwd Seg Size Min','Flow IAT Std','Flow Duration','Fwd IAT Mean', 'Total Fwd Packet', 'Total Bwd packets', 'Fwd IAT Max', 'Fwd IAT Std', 'Fwd IAT Total', 'Total Length of Bwd Packet','Bwd Header Length','Bwd Init Win Bytes','Fwd Act Data Pkts','Fwd Packet Length Max','Total Length of Fwd Packet','Packet Length Variance','Fwd Segment Size Avg','Bwd IAT Total','Packet Length Min','Bwd Segment Size Avg','Fwd Packet Length Min','Average Packet Size','Packet Length Max','Bwd Packet Length Max','Packet Length Std','Idle Std','Bwd IAT Mean','Bwd Packet Length Min','Bwd IAT Min','Bwd IAT Max']]
test_expt = pd.concat([test_expt, test_flags['Encoded_flags'],test_protocol,test_src_ip[['Class','CIDR']],test_dst_ip[['Class','CIDR']]], axis=1)
test_list = test_expt.values.tolist()

In [138]:
test_expt.head()

,Src Port,Flow IAT Mean,Flow IAT Max,Fwd Header Length,Dst Port,FWD Init Win Bytes,Fwd IAT Min,Flow IAT Min,Fwd Seg Size Min,Flow IAT Std,Flow Duration,Fwd IAT Mean,Total Fwd Packet,Total Bwd packets,Fwd IAT Max,Fwd IAT Std,Fwd IAT Total,Total Length of Bwd Packet,Bwd Header Length,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Packet Length Max,Total Length of Fwd Packet,Packet Length Variance,Fwd Segment Size Avg,Bwd IAT Total,Packet Length Min,Bwd Segment Size Avg,Fwd Packet Length Min,Average Packet Size,Packet Length Max,Bwd Packet Length Max,Packet Length Std,Idle Std,Bwd IAT Mean,Bwd Packet Length Min,Bwd IAT Min,Bwd IAT Max,Encoded_flags,Protocol,Class,CIDR,Class,CIDR
0,23801,1493537.0,1999032.0,120,53,29200,988042.0,988042.0,40,714877.884712,2987074,1493537.0,3,0,1999032.0,714877.884712,2987074.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000030,1,1,360340390,1,518985595
1,28413,1488972.0,1999057.0,120,53,29200,978887.0,978887.0,40,721369.124963,2977944,1488972.0,3,0,1999057.0,721369.124963,2977944.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000030,1,1,360340390,1,518985595
2,49550,1497687.5,2002658.0,120,53,29200,992717.0,992717.0,40,714136.129698,2995375,1497687.5,3,0,2002658.0,714136.129698,2995375.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000030,1,1,360340390,1,518985595
3,38373,1501817.5,2004628.0,120,53,29200,999007.0,999007.0,40,711081.428404,3003635,1501817.5,3,0,2004628.0,711081.428404,3003635.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000030,1,1,360340390,1,518985595
4,8805,1503194.0,2010151.0,120,53,29200,996237.0,996237.0,40,716945.464940,3006388,1503194.0,3,0,2010151.0,716945.464940,3006388.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000030,1,1,360340389,1,518985595


In [139]:
test_X = normalizer.transform(test_list)

In [140]:
test_X = test_X.astype(np.float32)

In [141]:
y_pred_test = model.predict(test_X)

In [142]:
print(y_pred_test.shape)

(325963,)


In [143]:
outer_test_all = open_pickle("./outer_IP_test_all.pkl")

In [144]:
print(len(outer_test_all))

1047


In [145]:
labelIP_dict_test = dict()

for ip in outer_test_all:
    labelIP_dict_test[ip] = 0


In [146]:
print(len(labelIP_dict_test))

1047


In [147]:
from tqdm import tqdm

predictIP_dict_test = dict()
external_IP =list(labelIP_dict_test.keys())
external_IP_group = dict()

train_df.rename(columns={'Src IP':'src_ip','Dst IP':'dst_ip'}, inplace=True)
valid_df.rename(columns={'Src IP':'src_ip','Dst IP':'dst_ip'}, inplace=True)
test_df.rename(columns={'Src IP':'src_ip','Dst IP':'dst_ip'}, inplace=True)

# grouping
for row in tqdm(test_df.itertuples()):
  src_ip = row.src_ip
  dst_ip = row.dst_ip

  if src_ip in external_IP:
    if src_ip in external_IP_group:
      external_IP_group[src_ip].append(row)
    else:
      external_IP_group[src_ip] = [row]
  else:
    if dst_ip in external_IP_group:
      external_IP_group[dst_ip].append(row)
    else:
      external_IP_group[dst_ip] = [row]

# if proportion 0.7
proportion = 0.6
for key, values in tqdm(external_IP_group.items()):
  predictIP_dict_test[key] = 0
  cnt = 0
  for value in values:
    if y_pred_test[value.Index] == 1:
      cnt += 1
      if cnt/float(len(values)) >= proportion:
          predictIP_dict_test[key] = 1
          break

325963it [00:03, 106664.53it/s]
100%|██████████| 1047/1047 [00:00<00:00, 2933.50it/s]


In [148]:
benign_count = 0
mal_count = 0

print("Total Count : ",len(predictIP_dict_test))

for key, value in predictIP_dict_test.items():
	if value == 0:
		benign_count += 1
	else:
		mal_count += 1
print("Benign count : ", benign_count)
print("Mal count : ", mal_count)

Total Count :  1047
Benign count :  585
Mal count :  462


In [149]:
import csv
csv_file = '03_project2_result.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)

    header = ["IP", "predict"]
    writer.writerow(header)

    for key, value in predictIP_dict_test.items():
        writer.writerow([key, value])